In [1]:
import evalml
from evalml import *
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *
from model_winner_select import *


from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

step = 1


In [3]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = train_keys,
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [4]:
load_from_disk = True
tec = TecAn(windows = 40, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(15, sourceDataGenerator, load_from_disk, 10)

In [5]:
trainX_raw, trainY_raw = online.load_train_data()


x, y = prepare_train_data(trainX_raw, trainY_raw, step)


print(Counter(y))

Counter({0: 23872, 1: 23872})


In [6]:

trainX_raw, trainY_raw = online.load_val_data("btcusd")
val_x, val_y = prepare_train_data(trainX_raw, trainY_raw, step)

print(Counter(val_y))

Counter({1: 2125, 0: 2125})


In [7]:
from tpot.config import classifier_config_dict
# add FeatureSetSelector into tpot configuration
classifier_config_dict['tpot.builtins.FeatureSetSelector'] = {
    'subset_list': ['https://raw.githubusercontent.com/EpistasisLab/tpot/master/tests/subset_test.csv'],
    'sel_subset': [0,1] # select only one feature set, a list of index of subset in the list above
    #'sel_subset': list(combinations(range(3), 2)) # select two feature sets
}



In [43]:
%%time

from tpot import TPOTClassifier

model = TPOTClassifier(generations=30, 
                       population_size=50, 
                       #scoring='f1', 
                       verbosity=2, 
                       #template='Transformer-Classifier',
                       #config_dict=classifier_config_dict,
                       random_state=1,
                       config_dict='TPOT sparse',
                       n_jobs=-1)
# perform the search
model.fit(x, y)

Optimization Progress:   0%|          | 0/1550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7812088448341526

Generation 2 - Current best internal CV score: 0.7812088448341526

Generation 3 - Current best internal CV score: 0.808185645156245

Generation 4 - Current best internal CV score: 0.808185645156245

Generation 5 - Current best internal CV score: 0.808185645156245

Generation 6 - Current best internal CV score: 0.808185645156245

Generation 7 - Current best internal CV score: 0.8088980884563263

Generation 8 - Current best internal CV score: 0.8432270800623846

Generation 9 - Current best internal CV score: 0.8467878816834459

Generation 10 - Current best internal CV score: 0.8467878816834459

Generation 11 - Current best internal CV score: 0.8467878816834459

Generation 12 - Current best internal CV score: 0.8581817471088092

Generation 13 - Current best internal CV score: 0.8630623372148409

Generation 14 - Current best internal CV score: 0.8630623372148409

Generation 15 - Current best internal CV score: 0.8630623372

TPOTClassifier(config_dict='TPOT sparse', generations=30, n_jobs=-1,
               population_size=50, random_state=1, verbosity=2)

In [44]:
print(model.score(val_x, val_y))

0.5084705882352941


In [45]:
val_keys = online.val_keys
for key in val_keys:
    print(key)
    back = test_model(model, key, online, step, verbose = False)
    back.report()

btcusd
8% -> 99.54120758328665
States: 2883 Positive: 65 Negative: 57
ethusd
10% -> 105.51913112386896
States: 2883 Positive: 87 Negative: 77
ltcusd
2% -> 85.0588067707423
States: 2883 Positive: 87 Negative: 85
ltcbtc
8% -> 99.70178363176986
States: 2883 Positive: 26 Negative: 18


In [46]:


#60% -> 121.58197499925188
#States: 2883 Positive: 157 Negative: 97


In [47]:
model.export('tpot_boston_pipeline.py')

In [13]:
automl = AutoMLSearch(X_train=x, y_train=y, problem_type='binary', 
                      max_batches=4,
                      objective='f1',
                      #additional_objectives=['auc', 'f1', 'precision'],
                      #additional_objectives=[ 'precision'],
                      ensembling=False)
automl.search()

Generating pipelines to search over...
7 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for F1. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 4 batches for a total of 23 pipelines. 
Allowed model families: linear_model, xgboost, decision_tree, random_forest, catboost, extra_trees



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean F1: 0.222
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Mode Baseline Binary Classification Pipeline may not perform as estimated on unseen data.

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean F1: 0.667
Decision Tree Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean F1: 0.676
Random Forest Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean F1: 0.685
CatBoost Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean F1: 0.667
Extra Trees Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean F1: 0.672
L

In [14]:
#automl.rankings

In [15]:
pipeline = automl.best_pipeline

#pipeline.graph()

eval_data(pipeline, val_x, val_y)


              precision    recall  f1-score   support

     class 0       0.52      0.83      0.64      2125
     class 1       0.58      0.22      0.32      2125

    accuracy                           0.53      4250
   macro avg       0.55      0.53      0.48      4250
weighted avg       0.55      0.53      0.48      4250



In [16]:
preds = pipeline.predict(val_x)

print(Counter(val_y))
print(Counter(preds))

Counter({1: 2125, 0: 2125})
Counter({0: 3423, 1: 827})


In [17]:
evalml.objectives.utils.get_core_objective_names()

['expvariance',
 'maxerror',
 'medianae',
 'mse',
 'mae',
 'r2',
 'root mean squared error',
 'mcc multiclass',
 'log loss multiclass',
 'auc weighted',
 'auc macro',
 'auc micro',
 'precision weighted',
 'precision macro',
 'precision micro',
 'f1 weighted',
 'f1 macro',
 'f1 micro',
 'balanced accuracy multiclass',
 'accuracy multiclass',
 'mcc binary',
 'log loss binary',
 'auc',
 'precision',
 'f1',
 'balanced accuracy binary',
 'accuracy binary']

In [18]:
back = test_model(pipeline, "btcusd", online, step, verbose = True)

2021-08-09 12:08:10.863695 BackTest: 1970-01-01 00:00:00 Buy (40386.42): ask: 40386.42
2021-08-09 12:08:10.901404 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.4156
2021-08-09 12:08:10.901536 BackTest: 1970-01-01 00:00:00 Sell (40218.56) profit: -0.4156 total: 99.58436523960282
2021-08-09 12:08:11.172430 BackTest: 1970-01-01 00:00:00 Buy (40281.3): ask: 40281.3
2021-08-09 12:08:11.218356 BackTest: 1970-01-01 00:00:00 Sell (40359.26) profit: 0.1935 total: 99.77790417702539
2021-08-09 12:08:11.390547 BackTest: 1970-01-01 00:00:00 Buy (40414.75): ask: 40414.75
2021-08-09 12:08:11.431709 BackTest: 1970-01-01 00:00:00 Profit detected price:40560.06  bid:40560.06 ask:40560.06
2021-08-09 12:08:11.431886 BackTest: 1970-01-01 00:00:00 Sell (40560.06) profit: 0.3595 total: 100.1374511246126
2021-08-09 12:08:11.473237 BackTest: 1970-01-01 00:00:00 Buy (40555.95): ask: 40555.95
2021-08-09 12:08:11.510658 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.2743
2021-08-09 12:08:11.510770 BackTest: 1970-0

2021-08-09 12:08:20.440326 BackTest: 1970-01-01 00:00:00 Profit detected price:37803.72  bid:37803.72 ask:37803.72
2021-08-09 12:08:20.440510 BackTest: 1970-01-01 00:00:00 Sell (37803.72) profit: 0.1431 total: 101.91307022527408
2021-08-09 12:08:20.677942 BackTest: 1970-01-01 00:00:00 Buy (37704.92): ask: 37704.92
2021-08-09 12:08:20.714629 BackTest: 1970-01-01 00:00:00 Sell (37792.06) profit: 0.2311 total: 102.14418064799875
2021-08-09 12:08:20.819978 BackTest: 1970-01-01 00:00:00 Buy (37820.67): ask: 37820.67
2021-08-09 12:08:20.853616 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.2131
2021-08-09 12:08:20.853743 BackTest: 1970-01-01 00:00:00 Sell (37740.08) profit: -0.2131 total: 101.93109610983484
2021-08-09 12:08:21.341183 BackTest: 1970-01-01 00:00:00 Buy (37414.57): ask: 37414.57
2021-08-09 12:08:21.375888 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.0498
2021-08-09 12:08:21.376110 BackTest: 1970-01-01 00:00:00 Sell (37395.92) profit: -0.0498 total: 101.8812492186371
2021-08-09 

2021-08-09 12:08:28.447973 BackTest: 1970-01-01 00:00:00 Buy (35792.55): ask: 35792.55
2021-08-09 12:08:28.483674 BackTest: 1970-01-01 00:00:00 Sell (35808.47) profit: 0.0445 total: 101.007636299253
2021-08-09 12:08:29.358722 BackTest: 1970-01-01 00:00:00 Buy (33911.28): ask: 33911.28
2021-08-09 12:08:29.393529 BackTest: 1970-01-01 00:00:00 Sell (33979.19) profit: 0.2003 total: 101.20789414855861
2021-08-09 12:08:29.428889 BackTest: 1970-01-01 00:00:00 Buy (33890.25): ask: 33890.25
2021-08-09 12:08:29.561338 BackTest: 1970-01-01 00:00:00 Sell (33919.48) profit: 0.0862 total: 101.29414314347603
2021-08-09 12:08:29.801384 BackTest: 1970-01-01 00:00:00 Buy (34168.64): ask: 34168.64
2021-08-09 12:08:29.838599 BackTest: 1970-01-01 00:00:00 Sell (34342.2) profit: 0.508 total: 101.8020942940047
2021-08-09 12:08:29.971482 BackTest: 1970-01-01 00:00:00 Buy (34400.87): ask: 34400.87
2021-08-09 12:08:30.006936 BackTest: 1970-01-01 00:00:00 Sell (34526.03) profit: 0.3638 total: 102.16592230184287


2021-08-09 12:08:36.095900 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -3.136
2021-08-09 12:08:36.096192 BackTest: 1970-01-01 00:00:00 Sell (30250.0) profit: -3.136 total: 101.73467746708154
2021-08-09 12:08:36.166468 BackTest: 1970-01-01 00:00:00 Buy (29403.73): ask: 29403.73
2021-08-09 12:08:36.200175 BackTest: 1970-01-01 00:00:00 Profit detected price:29897.14  bid:29897.14 ask:29897.14
2021-08-09 12:08:36.200333 BackTest: 1970-01-01 00:00:00 Sell (29897.14) profit: 1.6781 total: 103.41272987743899
2021-08-09 12:08:36.237340 BackTest: 1970-01-01 00:00:00 Buy (29810.0): ask: 29810.0
2021-08-09 12:08:36.270956 BackTest: 1970-01-01 00:00:00 Profit detected price:29852.64  bid:29852.64 ask:29852.64
2021-08-09 12:08:36.271146 BackTest: 1970-01-01 00:00:00 Sell (29852.64) profit: 0.143 total: 103.55576912601329
2021-08-09 12:08:36.306950 BackTest: 1970-01-01 00:00:00 Buy (29485.86): ask: 29485.86
2021-08-09 12:08:36.343446 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.8715
2021-08-09 12:08

2021-08-09 12:08:45.377201 BackTest: 1970-01-01 00:00:00 Buy (34375.16): ask: 34375.16
2021-08-09 12:08:45.414836 BackTest: 1970-01-01 00:00:00 Sell (34393.43) profit: 0.0531 total: 106.6645467301481
2021-08-09 12:08:45.451067 BackTest: 1970-01-01 00:00:00 Buy (34282.62): ask: 34282.62
2021-08-09 12:08:45.484855 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.5303
2021-08-09 12:08:45.484995 BackTest: 1970-01-01 00:00:00 Sell (34100.82) profit: -0.5303 total: 106.13424887076629
2021-08-09 12:08:45.521810 BackTest: 1970-01-01 00:00:00 Buy (34041.41): ask: 34041.41
2021-08-09 12:08:45.556169 BackTest: 1970-01-01 00:00:00 Profit detected price:34304.52  bid:34304.52 ask:34304.52
2021-08-09 12:08:45.556325 BackTest: 1970-01-01 00:00:00 Sell (34304.52) profit: 0.7729 total: 106.90716045110328
2021-08-09 12:08:45.592272 BackTest: 1970-01-01 00:00:00 Buy (34058.09): ask: 34058.09
2021-08-09 12:08:45.629908 BackTest: 1970-01-01 00:00:00 Sell (34083.89) profit: 0.0758 total: 106.98291337794093
202

...

2021-08-09 12:09:06.023830 BackTest: 1970-01-01 00:00:00 Buy (33154.76): ask: 33154.76
2021-08-09 12:09:06.061957 BackTest: 1970-01-01 00:00:00 Sell (33345.94) profit: 0.5766 total: 106.66573334756775
2021-08-09 12:09:06.433965 BackTest: 1970-01-01 00:00:00 Buy (33720.05): ask: 33720.05
2021-08-09 12:09:06.577688 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -1.5377
2021-08-09 12:09:06.577847 BackTest: 1970-01-01 00:00:00 Sell (33201.55) profit: -1.5377 total: 105.1280725196627
2021-08-09 12:09:06.689604 BackTest: 1970-01-01 00:00:00 Buy (33452.58): ask: 33452.58
2021-08-09 12:09:06.726495 BackTest: 1970-01-01 00:00:00 Profit detected price:33594.33  bid:33594.33 ask:33594.33
2021-08-09 12:09:06.726700 BackTest: 1970-01-01 00:00:00 Sell (33594.33) profit: 0.4237 total: 105.55180665317347
2021-08-09 12:09:06.795512 BackTest: 1970-01-01 00:00:00 Buy (33434.87): ask: 33434.87
2021-08-09 12:09:06.861473 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.3667
2021-08-09 12:09:06.861614 BackTest: 19

2021-08-09 12:09:18.268569 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.3993
2021-08-09 12:09:18.268738 BackTest: 1970-01-01 00:00:00 Sell (34501.59) profit: -0.3993 total: 109.05331511367346
2021-08-09 12:09:18.371603 BackTest: 1970-01-01 00:00:00 Buy (34363.72): ask: 34363.72
2021-08-09 12:09:18.471897 BackTest: 1970-01-01 00:00:00 Sell (34384.13) profit: 0.0594 total: 109.11270914901071
2021-08-09 12:09:18.579544 BackTest: 1970-01-01 00:00:00 Buy (34112.89): ask: 34112.89
2021-08-09 12:09:18.617459 BackTest: 1970-01-01 00:00:00 Sell (34200.0) profit: 0.2554 total: 109.36806716763651
2021-08-09 12:09:18.760854 BackTest: 1970-01-01 00:00:00 Buy (34358.06): ask: 34358.06
2021-08-09 12:09:18.798064 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.1835
2021-08-09 12:09:18.798305 BackTest: 1970-01-01 00:00:00 Sell (34295.01) profit: -0.1835 total: 109.18455855277294
2021-08-09 12:09:19.002962 BackTest: 1970-01-01 00:00:00 Buy (34174.91): ask: 34174.91
2021-08-09 12:09:19.043683 BackTest: 19

2021-08-09 12:09:26.245677 BackTest: 1970-01-01 00:00:00 Buy (34822.45): ask: 34822.45
2021-08-09 12:09:26.280379 BackTest: 1970-01-01 00:00:00 Sell (34859.24) profit: 0.1057 total: 111.81391642857011
2021-08-09 12:09:26.381743 BackTest: 1970-01-01 00:00:00 Buy (34733.9): ask: 34733.9
2021-08-09 12:09:26.421272 BackTest: 1970-01-01 00:00:00 Sell (34828.13) profit: 0.2713 total: 112.08520758792739
2021-08-09 12:09:27.538077 BackTest: 1970-01-01 00:00:00 Buy (34172.39): ask: 34172.39
2021-08-09 12:09:27.573339 BackTest: 1970-01-01 00:00:00 Sell (34263.85) profit: 0.2676 total: 112.35285055934378
2021-08-09 12:09:27.641676 BackTest: 1970-01-01 00:00:00 Buy (34176.34): ask: 34176.34
2021-08-09 12:09:27.675643 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.8092
2021-08-09 12:09:27.675787 BackTest: 1970-01-01 00:00:00 Sell (33899.8) profit: -0.8092 total: 111.54369428339383
2021-08-09 12:09:27.711524 BackTest: 1970-01-01 00:00:00 Buy (33989.31): ask: 33989.31
2021-08-09 12:09:27.780987 BackTe

2021-08-09 12:09:39.864021 BackTest: 1970-01-01 00:00:00 Buy (33976.16): ask: 33976.16
2021-08-09 12:09:39.899624 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.2048
2021-08-09 12:09:39.899772 BackTest: 1970-01-01 00:00:00 Sell (33906.59) profit: -0.2048 total: 110.27563600448751
2021-08-09 12:09:40.574957 BackTest: 1970-01-01 00:00:00 Buy (34472.37): ask: 34472.37
2021-08-09 12:09:40.613209 BackTest: 1970-01-01 00:00:00 Sell (34475.69) profit: 0.0096 total: 110.2852669059892
2021-08-09 12:09:40.650259 BackTest: 1970-01-01 00:00:00 Buy (34430.51): ask: 34430.51
2021-08-09 12:09:40.723289 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.0906
2021-08-09 12:09:40.723450 BackTest: 1970-01-01 00:00:00 Sell (34399.32) profit: -0.0906 total: 110.19467864575141
2021-08-09 12:09:41.816284 BackTest: 1970-01-01 00:00:00 Buy (34361.29): ask: 34361.29
2021-08-09 12:09:41.852212 BackTest: 1970-01-01 00:00:00 Sell (34398.21) profit: 0.1074 total: 110.30212513568237
2021-08-09 12:09:42.193930 BackTest: 19

2021-08-09 12:09:49.544139 BackTest: 1970-01-01 00:00:00 Buy (32882.52): ask: 32882.52
2021-08-09 12:09:49.579128 BackTest: 1970-01-01 00:00:00 Sell (32914.35) profit: 0.0968 total: 109.72596202027881
2021-08-09 12:09:49.861687 BackTest: 1970-01-01 00:00:00 Buy (32905.93): ask: 32905.93
2021-08-09 12:09:49.935331 BackTest: 1970-01-01 00:00:00 #### LOSSSS: -0.2755
2021-08-09 12:09:49.935502 BackTest: 1970-01-01 00:00:00 Sell (32815.29) profit: -0.2755 total: 109.450510148838
2021-08-09 12:09:50.006811 BackTest: 1970-01-01 00:00:00 Buy (32842.38): ask: 32842.38
2021-08-09 12:09:50.046587 BackTest: 1970-01-01 00:00:00 Sell (32892.56) profit: 0.1528 total: 109.60330053735431
2021-08-09 12:09:50.154024 BackTest: 1970-01-01 00:00:00 Buy (32690.65): ask: 32690.65
2021-08-09 12:09:50.188486 BackTest: 1970-01-01 00:00:00 Profit detected price:32695.21  bid:32695.21 ask:32695.21
2021-08-09 12:09:50.188644 BackTest: 1970-01-01 00:00:00 Sell (32695.21) profit: 0.0139 total: 109.61724947994188
2021

In [19]:
back.report()

60% -> 109.74456144790693
States: 2883 Positive: 218 Negative: 158
